# Packages, Imports, and Setup


In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
%cd /content/drive/My Drive/Machine Learning/Sketch2Graphviz

In [ ]:
%pip install -r requirements.txt

In [ ]:
import os
import random
import numpy as np
import torch
from dotenv import load_dotenv
from huggingface_hub import login
import matplotlib.pyplot as plt

from scripts.data import get_json_graphviz_json_dataloaders, get_graphviz_hf_dataloaders
from scripts.model import (
    Sketch2GraphvizVLM,
    save_sketch2graphviz_vlm_local,
    load_sketch2graphviz_vlm_local,
    print_num_params,
)
from scripts.finetune_lora import finetune_vlm_lora
from scripts.eval import evaluate_vlm

from scripts.inference import predict_graphviz_dot

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

load_dotenv()
hf_token = os.getenv("HF_TOKEN")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
login(token=hf_token)

In [ ]:
!nvidia-smi

In [ ]:
batch_size = 1

instruction = """
You are an expert compiler that converts images of Graphviz diagrams into their exact Graphviz DOT code.
Given an image of a graph, using only the image, output only the DOT code, starting with either 'digraph' or 'graph', with no explanations, no markdown, and no extra text.
Graphviz DOT is a plain-text language for describing graphs as nodes and edges with optional attributes such as labels, shapes, colors, and styles, for both directed ('digraph') and undirected ('graph') diagrams.

## Core Syntax Rules

1.  **Graph Type:**
      * Use `digraph` (Directed Graph) for hierarchies, flows, or dependencies. Use `->` for edges.
      * Use `graph` (Undirected Graph) for physical networks or mutual connections. Use `--` for edges.
2.  **Identifiers:**
      * Alphanumeric strings (e.g., `A`, `node1`) do not need quotes.
      * Strings with spaces, special characters, or reserved keywords MUST be enclosed in double quotes (e.g., `"User Login"`, `"Data-Base"`).
3.  **Statement Termination:** End all node, edge, and attribute statements with a semicolon `;`.
4.  **Scope:** All code must be enclosed within braces `{ ... }`.

## Attribute Dictionary

Apply attributes using brackets `[key=value]`. If multiple attributes are needed, comma-separate them or use spaces: `[shape=box, color=red]`.

### Node Attributes

  * **`shape`**:
      * Process/Step: `box`
      * Start/End: `ellipse` or `oval`
      * Decision: `diamond`
      * Database: `cylinder`
      * Code/Structure: `record` (use `|` to separate fields in label)
  * **`style`**: `filled`, `rounded`, `dotted`, `invis`
  * **`fillcolor`**: Hex codes (`#FF0000`) or common names (`lightblue`). Only visible if `style=filled`.
  * **`label`**: The visible text. If omitted, the identifier is used.

### Edge Attributes

  * **`label`**: Text displayed along the line.
  * **`style`**: `solid` (default), `dashed` (future/theoretical), `dotted`.
  * **`dir`**: `forward` (default), `back`, `both`, `none`.
  * **`color`**: Edge color.

  * Output **only** the code block.
  * Do not include any explanations.
  * Ensure all braces `{}` are balanced.

"""

# Loading Data


In [ ]:
# train_dataloader, val_dataloader, test_dataloader = get_graphviz_hf_dataloaders(
#     batch_size=batch_size,
#     root_dir="graphviz_rendered",
#     image_size=(768, 768),  # (1024, 1024)
# )

train_dataloader, test_dataloader = get_json_graphviz_json_dataloaders(
    json_path="simple_synthetic_data_gen.json",
    batch_size=batch_size,
    root_dir="graphviz_rendered_json",
    image_size=(768, 768),  # (1024, 1024)
)

In [ ]:
train_sample = next(iter(train_dataloader))

plt.imshow(np.array(train_sample["images"][0]))
plt.axis(False)
plt.show()

print(f"Graphviz Code: {train_sample["graphviz_code"][0]}")

# Loading Sketch2Graphviz Model


In [ ]:
model = Sketch2GraphvizVLM(
    llama_model_id="meta-llama/Llama-3.2-11B-Vision-Instruct",
    quantization="16-bit",
    device=device,
)

if model.quantization != "16-bit":
    model.llama_model.gradient_checkpointing_enable()
    model.llama_model.config.use_cache = False
    model.llama_model.enable_input_require_grads()

print_num_params(model)

# LoRA Fine-Tuning and Evaluation


In [ ]:
lora_rank = 32
lora_dropout = 0.1

lr_lora = 2e-4  # 1e-4
weight_decay_lora = 1e-3  # 1e-2
max_grad_norm = 1.0

num_epochs = 10

model, train_losses, val_losses = finetune_vlm_lora(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=test_dataloader,
    instruction=instruction,
    rank=lora_rank,
    lora_dropout=lora_dropout,
    lr_lora=lr_lora,
    weight_decay_lora=weight_decay_lora,
    num_epochs=num_epochs,
    use_val_early_stopping=True,
    max_grad_norm=max_grad_norm,
    model_save_dir="checkpoints",
    device=device,
)

print(f"Train Losses: {train_losses}")
print(f"Val Losses: {val_losses}")

In [ ]:
test_loss = evaluate_vlm(
    model=model,
    iterator=test_dataloader,
    instruction=instruction,
    description="Testing",
    model_load_dir="checkpoints",
    epoch_load=None,
    device=device,
)

print(f"Test Loss: {test_loss:.6f}")

In [ ]:
best_epoch = 10

save_sketch2graphviz_vlm_local(
    model=model, model_save_dir="checkpoints", epoch_save=best_epoch
)

model = load_sketch2graphviz_vlm_local(
    model_load_dir="checkpoints",
    epoch_load=best_epoch,
    quantization="16-bit",
    device=device,
)

# Inference


In [ ]:
predicted_graphviz_output = predict_graphviz_dot(
    model=model,
    image="testing_graphs/graph_4.png",
    instruction=instruction,
    max_new_tokens=2048,
    do_sample=False,
    temperature=0.3,
    skip_special_tokens=False,
    device=device,
)

print(predicted_graphviz_output)

# Clip-Llama OLD Model


In [ ]:
# model = CLIPLlamaSketch2GraphvizVLM(
#     vit_model_id="openai/clip-vit-large-patch14-336",
#     llama_model_id="meta-llama/Llama-3.1-8B-Instruct",
#     quantization="4-bit",
#     tile_images=False,
#     use_cross_attention=False,
#     device=device,
# ).to(device)

# if model.quantization != "16-bit":
#     model.llama_model.gradient_checkpointing_enable()
#     model.llama_model.config.use_cache = False
#     model.llama_model.enable_input_require_grads()

# print_num_params(model)

# lora_rank = 32
# lora_dropout = 0.1

# lr_vit = 1e-5  # 5e-6
# lr_proj = 1e-4  # 5e-5
# lr_cross_attention = 1e-4  # 5e-5
# lr_lora = 2e-4  # 1e-4

# weight_decay_vit = 1e-2  # 5e-2
# weight_decay_proj = 1e-2  # 5e-2
# weight_decay_cross_attention = 1e-2  # 5e-2
# weight_decay_lora = 1e-3  # 1e-2

# max_grad_norm = 1.0

# num_epochs = 10

In [ ]:
# model, train_losses, val_losses = train_projection_cross_attention(
#     model=model,
#     train_dataloader=train_dataloader,
#     val_dataloader=test_dataloader,
#     instruction=instruction,
#     lr_proj=lr_proj,
#     lr_cross_attention=lr_cross_attention,
#     weight_decay_proj=weight_decay_proj,
#     weight_decay_cross_attention=weight_decay_cross_attention,
#     num_epochs=5,
#     use_val_early_stopping=True,
#     max_grad_norm=max_grad_norm,
#     device=device,
# )

# print(f"Train Losses: {train_losses}")
# print(f"Val Losses: {val_losses}")

In [ ]:
# model = add_lora_to_llama(
#     model, rank=lora_rank, alpha=(2 * lora_rank), dropout=lora_dropout
# )
# model = add_lora_to_vit(
#     model, rank=lora_rank, alpha=(2 * lora_rank), dropout=lora_dropout
# )
# print_num_params(model)

In [ ]:
# model, train_losses, val_losses = finetune_vlm_lora(
#     model=model,
#     train_dataloader=train_dataloader,
#     val_dataloader=test_dataloader,
#     instruction=instruction,
#     rank=lora_rank,
#     lora_dropout=lora_dropout,
#     lr_vit=lr_vit,
#     lr_proj=lr_proj,
#     lr_cross_attention=lr_cross_attention,
#     lr_lora=lr_lora,
#     weight_decay_vit=weight_decay_vit,
#     weight_decay_proj=weight_decay_proj,
#     weight_decay_cross_attention=weight_decay_cross_attention,
#     weight_decay_lora=weight_decay_lora,
#     num_epochs=num_epochs,
#     use_val_early_stopping=True,
#     max_grad_norm=max_grad_norm,
#     model_save_dir="checkpoints",
#     device=device,
# )

# print(f"Train Losses: {train_losses}")
# print(f"Val Losses: {val_losses}")

In [ ]:
# test_loss = evaluate_clip_llama_vlm(
#     model=model,
#     iterator=test_dataloader,
#     instruction=instruction,
#     description="Testing",
#     model_load_dir="checkpoints",
#     epoch_load=None,
#     device=device,
# )

# print(f"Test Loss: {test_loss:.6f}")

In [ ]:
# save_sketch2graphviz_clip_llama_vlm_local(model=model, model_save_dir="checkpoints", epoch_save=6)

# model = load_sketch2graphviz_clip_llama_vlm_local(
#     model_load_dir="checkpoints",
#     epoch_load=6,
#     device=device,
# )

In [ ]:
# import torch
# from PIL import Image
# from transformers import MllamaForConditionalGeneration, AutoProcessor

# testing_model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

# testing_model = MllamaForConditionalGeneration.from_pretrained(
#     testing_model_id,
#     torch_dtype=torch.float16,
#     device_map="auto",
# )
# processor = AutoProcessor.from_pretrained(testing_model_id)

In [ ]:
# image = Image.open("testing_graphs/graph_4.png").convert("RGB")

# messages = [
#     {
#         "role": "user",
#         "content": [{"type": "image"}, {"type": "text", "text": instruction}],
#     }
# ]
# input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
# inputs = processor(image, input_text, add_special_tokens=False, return_tensors="pt").to(
#     testing_model.device
# )

# output = testing_model.generate(**inputs, max_new_tokens=2048, do_sample=False)
# print(processor.decode(output[0]))